This is the demo of How to use the SQLalchemy to connect to the database and do the query.

First of all, you need to install the SQLalchemy package.

```bash
pip install sqlalchemy
```

Then, you need to install the database driver. For example, if you want to connect to the MySQL database, you need to install the MySQL driver.

```bash
pip install pymysql
```

# 1. Getting the connection to the database

For the testing purpose, we will use the demo database called 'testing-demo'. There is a demo table called 'users' in the database. The table has the following columns:

- id: the primary key of the table
- first_name: the first name of the user
- last_name: the last name of the user
- age: the age of the user
- email: the email of the user
- gender : the gender of the user
- ip_address: the ip address of the user

In [ ]:
# Import the function from database.py which will return the engine of the database
from database import get_database_engine

# For the get_database_engine function to work, you need to login the data-storage sso in the aws cli.
# You can do this by running the following command in the terminal:
# aws sso configure

engine_result = get_database_engine(database_name="testing-demo")

engine = engine_result.ok_value

# 2. Query the database

## 2.1 sqlalchemy support the sql query directly.

For sqlalchemy, we must connect the database in term of context. So, we need to use the `with` statement to connect to the database.

For utilizing the power of ORM, we will using the session class in sqlalchemy. The session class is the interface to the database. We can use the session class to query the database.

In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    result = session.execute("SELECT * FROM users")
    print(result.fetchall())

In [ ]:
with Session(engine) as session:
    result = session.execute("SELECT * FROM users")
    print(result.fetchone())

In [ ]:
with Session(engine) as session:
    result = session.execute(
        "SELECT * FROM users where gender like 'male' and age = 20"
    )
    print(result.first())

## 2.2 Create a table object

Create a table object for sqlalchemy can help us to avoid using the sql query directly. We can use the table object to query the database.

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata_obj = MetaData()

user_table = Table(
    "users",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("first_name", String),
    Column("last_name", String),
    Column("age", Integer),
    Column("email", String),
    Column("gender", String),
    Column("ip_address", String),
)

In [ ]:
with Session(engine) as session:
    result = session.execute(user_table.select())
    print(result.fetchall())

With the tool provided by sqlalchemy, we can first create a statement as a object. Then, we can use the object to query the database. 

In [ ]:
from sqlalchemy import select

statement = (
    select(user_table)
    .where(user_table.c.age == 20)
    .order_by(user_table.c.first_name.desc())
)

with Session(engine) as session:
    result = session.execute(statement).all()
    print(result)

We can also have some aggregate functions with GROUP BY / HAVING.

In [ ]:
from sqlalchemy import func

statement = select(
    user_table.c.gender, func.count(user_table.c.id).label("count")
).group_by(user_table.c.gender)

with Session(engine) as session:
    result = session.execute(statement).fetchall()

    # Iterate and print results
    for gender, count in result:
        print(f"Gender: {gender}, Count: {count}")

# 3. Insert the data into the database

In [ ]:
from sqlalchemy import insert

user_info = {
    "id": 1001,
    "first_name": "Ken",
    "last_name": "Chan",
    "age": 23,
    "email": "abc@gmail.com",
    "gender": "Male",
    "ip_address": "192.168.1.1",
}

with Session(engine) as session:
    session.execute(insert(user_table), [user_info])
    session.commit()